In [4]:
import PySimpleGUI as sg
import modulo as m
import os


BD = []

# ------------------------------------- JANELA EXTRA -------------------------------------

def janelaExtra(evento):
    if evento=="Listar por ator":
        exemplo="Ex: \"Selena Gomez\""           #Para as pessoas saberem o exemplo a utilizar
        procura="Insira o nome do ator que pretende procurar:"
    elif evento=="Listar por género":
        exemplo="Ex: \"Action\""
        procura="Insira o género que pretende procurar:"
    elif evento=="Consultar filme":
        exemplo="Ex: \"Dog\""
        procura="Insira uma palavra que pretende procurar:"
    layout2=[[sg.Text(procura,background_color='#1A6E74')], [sg.InputText(exemplo, key='-PROCURA-'), sg.Button("Pesquisar" ,button_color=('white', '#6ABDC3'))]]
    window2= sg.Window(title="Janela de pesquisa", font="Arial 20",background_color='#1A6E74', default_element_size=(25,1)).Layout(layout2)

    stop=False  
    while not stop:
        event, values = window2.read()
        if event == sg.WIN_CLOSED or event == "Pesquisar":
            stop = True

    window2.close()
    return values["-PROCURA-"]                # DEVOLVE A PESQUISA QUE O UTILIZADOR QUER EFETUAR



# ------------------------------------- MENSAGEM DE ERRO -------------------------------------

def janelaErro(mensagem):
    layout=[[sg.Text(mensagem,background_color='#1A6E74')], [sg.Button("Ok", button_color=('white', '#6ABDC3'))]]
    window= sg.Window(title="Mensagem", font="Arial 20",background_color='#1A6E74', default_element_size=(15,1)).Layout(layout)

    stop=False  
    while not stop:
        event, values = window.read()
        if event == "Ok" or event == sg.WIN_CLOSED:
            stop = True

    window.close()

# ------------------------------------- INSERIR NOVO FILME  -------------------------------------

def novoFilme():
    layout3=[[sg.Text("Titulo:",background_color='#1A6E74'), sg.InputText("", key='-TITULO-', size=(32))],
            [sg.Text("Ano:",background_color='#1A6E74'), sg.InputText("", key='-ANO-', size=(33))],
            [sg.Text("Elenco:",background_color='#1A6E74'), sg.InputText("", key='-ELENCO-', size=(31))],
            [sg.Text("Género:",background_color='#1A6E74'), sg.InputText("", key='-GÉNERO-', size=(30))],
            [sg.Button("Adicionar", button_color=('white', '#6ABDC3'))]]
            

    window3= sg.Window(title="Inserir novo filme", font="Arial 20", background_color='#1A6E74', default_element_size=(30)).Layout(layout3)

    stop=False  
    while not stop:
        event, values = window3.read()
        if event == "Ok" or event == sg.WIN_CLOSED:
            stop = True

    window3.close()
    filme = m.novofilme(values['-TITULO-'],values['-ANO-'], values['ELENCO-'],values['-GÉNERO-'])
    print(filme)


# ------------------------------------- INTERFACE PRINCIPAL -------------------------------------

listbox=[]
layout = [[sg.Column([[sg.Button("Carregar BD", button_color=('white', '#6ABDC3')), sg.Button("Guardar BD", button_color=('white', '#6ABDC3'))],
            [sg.InputText("", key='-DADOS-', size=(23,1))],
            [sg.Button("Listar filmes alfabeticamente", button_color=('white', '#6ABDC3'))],
            [sg.Button("Listar por género", button_color=('white', '#6ABDC3'))],
            [sg.Button("Listar por ator", button_color=('white', '#6ABDC3'))],
            [sg.Button("Consultar filme", button_color=('white', '#6ABDC3'))],
            [sg.Button("Filmes na BD", button_color=('white', '#6ABDC3'))],
            [sg.Button("Inserir novo filme", button_color=('white', '#6ABDC3'))], 
            [sg.Button("Distribuição por género", button_color=('white', '#6ABDC3'))],
            [sg.Button("Distribuição por ator", button_color=('white', '#6ABDC3'))],
            [sg.Button("Sair", button_color=('white', '#6ABDC3'))],            
            [sg.Button("?", button_color=('white', '#6ABDC3'))]],background_color='#1A6E74'), sg.Listbox(values=listbox, size=(35,22), key='-LIST-')]]



window=sg.Window(title="Cinemateca", font="Arial 20", default_element_size=(15,1),background_color='#1A6E74').Layout(layout)
modoajuda=False
stop=False  
while not stop:
    event, values = window.read()

    if event=="Carregar BD":
        if os.path.exists(values['-DADOS-']):         # VERIFICA SE O FICHEIRO EXISTE  #BFE9A2
            BD= m.lerficheiro(values['-DADOS-'])
            if BD==None:
                janelaErro("Introduza um ficheiro com extensão válida!") 
            else:
                janelaErro("BD carregada com sucesso!")
        else:
            janelaErro("Não foi encontrada nenhuma base de dados com esse nome!")

    elif event=="Guardar BD":
        if values['-DADOS-'].replace(" ", "")== '':              # verifica se o utilizador nao inseriu apenas espaços
            janelaErro("Introduza um nome para a nova BD!") 
        elif os.path.isfile(values['-DADOS-']):                  # verifica se o utilizador inseriu o nome de um ficheiro ja existente
            janelaErro("Já existe um ficheiro com esse nome!") 
        else:
            values['-DADOS-'] = m.tiraext(values['-DADOS-'])
            if modoajuda:
                janelaErro("Introduza o nome do ficheiro em que pretende guardar a base de dados na caixa de texto, \n preferencialmente sem extensão (será sempre guardado em json)")
                modoajuda =False
            else:
                    m.guardarficheiro(values['-DADOS-'],BD)

    elif event=="Listar filmes alfabeticamente":
        if BD != []:
            listbox=m.verfilmes(BD)
            listbox.sort()
            window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")
        

    elif event=="Listar por ator" or event=="Listar por género" or event=="Consultar filme":
        if BD != []:
            pesquisa=janelaExtra(event)                   # FAZER CÓDIGO QUE VAI TRABALHAR COM A PALAVRA INSERIDA
            if event=="Listar por ator":                  # E DEVOLVER A RESPOSTA NA LIST BOX 
                listbox=m.listarFilmeDeAtor(BD, pesquisa)
                window['-LIST-'].Update(values=listbox)
            if event=="Listar por género":
                listbox=m.listarGenero(BD, pesquisa)
                window['-LIST-'].Update(values=listbox)
            if event =='Consultar filme':
                filmes = m.verfilmes(BD)
                listbox = m.consultarfilme(filmes,pesquisa)
                window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")

    elif event=="Filmes na BD":
        listbox=m.contaFilmes(BD)
        window['-LIST-'].Update(values=listbox)

    elif event=="Inserir novo filme":
        listbox=novoFilme()
        window['-LIST-'].Update(values=listbox)
    
    elif event=="Distribuição por género":
        if BD != []:
            listbox=m.dictolist(m.distribuicao(BD,'genres'))
            window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")
    elif event=="Distribuição por ator":
        if BD!=[]:
            listbox=m.dictolist(m.distribuicao(BD,'cast'))
            window['-LIST-'].Update(values=listbox)
        else:
            janelaErro("Nenhuma BD carregada")

    elif event=="?":
        modoajuda=True

    if event == "Sair" or event == sg.WIN_CLOSED:
        stop = True

window.close()






TypeError: 'NoneType' object is not subscriptable